## Initialization of environemnt, imports

In [1]:
#from __future__ import absolute_import
#from __future__ import print_function
#from __future__ import division

from IPython.display import display, HTML

from importlib import reload

import pandas as pd
import json, os, re, shutil, sys, time
import multiprocessing as mp
from multiprocessing import Queue
from multiprocessing import Pool
import collections, itertools
import unittest
import datetime
import string
import pickle
import string
import copy

from cytoolz import concatv

# NLTK for NLP utils and corpora
import nltk,pprint
from nltk import word_tokenize
nltk.download('treebank')
nltk.download('punkt')

# NumPy and TensorFlow
import numpy as np
import tensorflow as tf
assert(tf.__version__.startswith("1."))

#parameter setting
work_dir = "working_dir/"

[nltk_data] Downloading package treebank to
[nltk_data]     /Users/i812749/nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package punkt to /Users/i812749/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def sent_tokenize_rm_punct (blob):
    #input is string
    #remove punctuations
    #change all to lower case
    #change 0-9 to N (we keep the number of digits so 321 will be NNN this way system will try to generate NNN in the approximate magnitute)
    #output is a list of strings (using ,?!, as senence spliter and split the sentence)
    bexp=''
    blob.replace('\n','')
    #blob.replace('</p>','  ')

    for i, char in (enumerate(blob)):
        #print(i,char)
        next_cap = False
        prev_lower = False
        next_space = False
        
        if char in string.punctuation :
            #print("found punctuation:", i, char, blob[i+1])
           
            if char in '.?!' :
                if i+1 >= len(blob):
                    next_cap = False
                else:
                    next_cap = blob[i+1].isupper()
                    next_space = blob[i+1].isspace()    


                if i-1 <0:
                    prev_lower = False
                else:
                    prev_lower = blob[i-1].islower()

                if (next_cap and prev_lower) or next_space : 
                    
                    #mark "period" here as this is likely real end of sentence.
                    bexp=bexp + ' <p> '
                else:
                    #if this is not end of paragraph, then 
                    #bexp = bexp + char
                    pass
            else :
                bexp = bexp + ' '
        elif char.isspace() :
            if i-1 >=0 :
                if blob[i-1].isspace() :
                    pass
                else :
                    bexp = bexp + char.lower()
        elif char == '\n':
            pass

        else :
            if char.isnumeric() : 
                bexp = bexp + 'N'
            else :
                bexp = bexp + char.lower()
    
    return(bexp)




In [23]:
class NYTData:
    __author__ = "Alan Tan"
    __copyright__ = "Copyright 2018, Alan Tan"
    __credits__ = ["Alan Tan"]
    __license__ = "Apache"
    __version__ = "2.0"
    __updatedby__ = "Alan Tan, 2018"
    
    def __init__(self, ):
        self.source_raw=[]
        self.target_raw=[]
        self.source_token_ids=[]
        self.target_token_ids=[]
        self.vocab_to_int={'<PAD>': 0, '<EOS>': 1, '<UNK>': 2, '<GO>': 3 , '<p>': 4}
        self.int_to_vocab={0 : '<PAD>', 1: '<EOS>', 2: '<UNK>', 3: '<GO>', 4 : '<p>'}

        
    def load_from_file(self, input_file_path='nyt_dataset/abstract_nyt_structured_data.csv'):
        #read file into Pandas Dataframe.
        #each line is one record, it contains "original file name(including y/m/d)","TITLE","ABSTRACT","LEAD_PARAGRAPH","FULL_TEXT"
        nyt_data = pd.read_csv(input_file_path, 
                               delimiter=',', index_col=0, header=None, quotechar='"', quoting=1, 
                               skipinitialspace=True, engine='c')


        #We will use LEAD_PARAGRAPH as source for now
        #And ABSTRACT as target. 
        # Remove punctuation, change digits to NNN, and to lower case.     
        self.source_raw=nyt_data[3]
        self.target_raw=nyt_data[2]
        print("Read",len(self.source_raw),"records into source text.")
        print("Read",len(self.target_raw),"records into target text.")
        
    def preprocess(self, max_sentence_length=np.inf):
        # make a list of unique words
        CODES = {'<PAD>': 0, '<EOS>': 1, '<UNK>': 2, '<GO>': 3 }

        self.source_token_ids = []
        self.target_token_ids = []
        
        pool = Pool()
        source_text = pool.map(sent_tokenize_rm_punct, self.source_raw)
        pool.close()
        pool.join()        
        # serial mode, no parallel multiprocessing .# source_text= sum([sent_tokenize_rm_punct(sentlst) for sentlst in self.source_raw],[])
        print("source_text length" , len(source_text))
        
        pool = Pool()
        target_text = pool.map(sent_tokenize_rm_punct, self.target_raw)
        pool.close()
        pool.join()
        #target_text= sum([sent_tokenize_rm_punct(sentlst) for sentlst in self.target_raw],[])
        print("target_text length", len(target_text)) 

        
        source_short_text,target_short_text=zip(*([(st, tt) for st, tt, in zip(source_text, target_text) if len(st.split())<=max_sentence_length*2 and len(tt.split())<=max_sentence_length]))
        source_short_length = [len(tt.split()) for tt in source_short_text]
        target_short_length = [len(tt.split()) for tt in target_short_text]
        print("only", len(source_short_text), "source sentences are less than",max(source_short_length), "long.")
        print("only", len(target_short_text), "target sentences are less than",max(target_short_length), "long.")
        
        
        #This is a summary model, so we create *one* vocab, (unlike translate model, where 2 separate vocab would be needed)
        vocab = set(' '.join(source_text).split()).union(' '.join(target_text).split())
        print("vocab size", len(vocab))

        # initially vocab starts with the special tokens, and we build all words into the dict vocab_to_int
        self.vocab_to_int = copy.copy(CODES)
        for w_i, w in enumerate(vocab, len(CODES)):
            self.vocab_to_int[w] = w_i
            
        # create reverse lookup dict(from index to word)
        self.int_to_vocab = {w_i: w for w, w_i in self.vocab_to_int.items()}

        print("vocab dict built:",len(self.vocab_to_int))
        
        # create token_ids which are intger indexes of words in our vocab (after tokenization and lower case)
        for source_sentence in source_short_text :
            source_token_id = [self.vocab_to_int[s] for s in source_sentence.split()]
            self.source_token_ids.append(source_token_id)
        print("source_token_ids size:",len(self.source_token_ids))
            
        for target_sentence in target_short_text :
            target_token_id = [self.vocab_to_int[s] for s in target_sentence.split()]
            self.target_token_ids.append(target_token_id)        
        print("target_token_ids size:",len(self.target_token_ids))
        

    def infer_prep(self,infer_sentences, pad=True):
        sentences_int=[]
        max_sentence_length=0
        for sentence in [sent_tokenize_rm_punct(sent) for sent in infer_sentences] :
            
            sentence_int=[]
            for word in sentence.split() :
                if word in self.vocab_to_int :
                    sentence_int.append(self.vocab_to_int[word])
                else :
                    sentence_int.append(self.vocab_to_int['<UNK>'])
            sentences_int.append(sentence_int)
            max_sentence_length=max(max_sentence_length,len(sentence_int))
            
        if pad :

            """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
            output_ids = [sentence_int + [self.vocab_to_int['<PAD>']] * (max_sentence_length - len(sentence_int)) for sentence_int in sentences_int]
        else :
            output_ids = sentences_int
        return output_ids


    def get_batches(self, batch_size):
        """Batch targets, sources, and the lengths of their sentences together"""
        for batch_i in range(0, len(self.source_token_ids)//batch_size):
            start_i = batch_i * batch_size

            # Slice the right amount for the batch
            sources_batch = self.source_token_ids[start_i:start_i + batch_size]
            targets_batch = self.target_token_ids[start_i:start_i + batch_size]

            # Pad
            max_sentence = max([len(sentence) for sentence in sources_batch])
            pad_sources_batch = np.array([sentence + [self.vocab_to_int['<PAD>']] * (max_sentence - len(sentence)) for sentence in sources_batch])
            max_sentence = max([len(sentence) for sentence in targets_batch])
            pad_targets_batch = np.array([sentence + [self.vocab_to_int['<PAD>']] * (max_sentence - len(sentence)) for sentence in targets_batch])

            # Need the lengths for the _lengths parameters
            pad_targets_lengths = []
            for target in pad_targets_batch:
                pad_targets_lengths.append(len(target))

            pad_sources_lengths = []
            for source in pad_sources_batch:
                pad_sources_lengths.append(len(source))

            yield pad_sources_batch, pad_targets_batch, pad_sources_lengths, pad_targets_lengths


        
    def save_preprocess(self, save_file_name='preprocess.p'):    
        # Save data for later use
        pickle.dump(self, open(save_file_name, 'wb'))
        
        print("Saved (", len(self.source_token_ids),  len(self.target_token_ids) , ") source , target tokens")
        print("   ...and (",len(self.vocab_to_int),") voc-int dictionary")
        print("   ...and (",len(self.int_to_vocab),") int-voc dictionary")
    
    def load_preprocess(self, load_file_name='preprocess.p'):
        with open(load_file_name, mode='rb') as in_file:
            
            self=pickle.load(in_file)        
        
    

In [24]:
if __name__ == '__main__':
    from_scratch=False
    save_file=work_dir+'short_preprocess.p'
    NYTDATA=NYTData()
    
    if from_scratch :
        NYTDATA.load_from_file('nyt_dataset/short_abstract_nyt_structured_data.csv')
        NYTDATA.preprocess(max_sentence_length=50)
    
        with open(save_file, 'wb') as f:
            pickle.dump(NYTDATA, f)
    else : #to load back
        with open(save_file, 'rb') as f:
            NYTDATA = pickle.load(f)

In [5]:

#with open(work_dir+'short_preprocess.p', 'rb') as f:
#        NYTDATA = pickle.load(f)
#max([len(text) for text in NYTDATA.source_token_ids])

## Routines to build Graph

In [6]:
def enc_dec_model_inputs():
    inputs = tf.placeholder(tf.int32, [None, None], name='input')
    targets = tf.placeholder(tf.int32, [None, None], name='targets') 
    
    target_sequence_length = tf.placeholder(tf.int32, [None], name='target_sequence_length')
    max_target_len = tf.reduce_max(target_sequence_length)    
    
    return inputs, targets, target_sequence_length, max_target_len


In [7]:
def process_decoder_input(target_data, target_vocab_to_int, batch_size):
    # get '<GO>' id
    go_id = target_vocab_to_int['<GO>']
    
    after_slice = tf.strided_slice(target_data, [0, 0], [batch_size, -1], [1, 1])
    after_concat = tf.concat( [tf.fill([batch_size, 1], go_id), after_slice], 1)
    
    return after_concat

In [8]:
def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob, 
                   source_vocab_size, 
                   encoding_embedding_size):
    """
    :return: tuple (RNN output, RNN state)
    """
    embed = tf.contrib.layers.embed_sequence(rnn_inputs, 
                                             vocab_size=source_vocab_size, 
                                             embed_dim=encoding_embedding_size)
    
    stacked_cells = tf.contrib.rnn.MultiRNNCell([tf.contrib.rnn.DropoutWrapper(tf.contrib.rnn.LSTMCell(rnn_size), keep_prob) for _ in range(num_layers)])
    
    outputs, state = tf.nn.dynamic_rnn(stacked_cells, 
                                       embed, 
                                       dtype=tf.float32)
    return outputs, state

In [9]:
def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, 
                         target_sequence_length, max_summary_length, 
                         output_layer, keep_prob):
    """
    Create a training process in decoding layer 
    :return: BasicDecoderOutput containing training logits and sample_id
    """

    #decoder_initial_state = dec_cell.zero_state(batch_size, tf.float32).clone(cell_state=encoder_state)
    #decoder_initial_state = encoder_state

    #print("decoder_init_state is type:", type(decoder_initial_state))
    #print(decoder_initial_state)
    # for only input layer
    helper = tf.contrib.seq2seq.TrainingHelper(dec_embed_input, 
                                               target_sequence_length)
    
    decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell, 
                                              helper, 
                                              initial_state=encoder_state, 
                                              output_layer=output_layer)

    # unrolling the decoder layer
    outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(decoder, 
                                                      impute_finished=True, 
                                                      maximum_iterations=max_summary_length)
    return outputs

In [10]:
def decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id,
                         end_of_sequence_id, max_target_sequence_length,
                         vocab_size, output_layer, batch_size, keep_prob):
    """
    Create a inference process in decoding layer 
    :return: BasicDecoderOutput containing inference logits and sample_id
    """
    
    #decoder_initial_state = encoder_state
    #decoder_initial_state = dec_cell.zero_state(batch_size, tf.float32).clone(cell_state=encoder_state)

    
    #dec_cell = tf.contrib.rnn.DropoutWrapper(dec_cell, 
    #                                         output_keep_prob=keep_prob)
    
    helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(dec_embeddings, 
                                                      tf.fill([batch_size], start_of_sequence_id), 
                                                      end_of_sequence_id)
    decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell, 
                                              helper, 
                                              initial_state=encoder_state, 
                                              output_layer=output_layer)
    
    outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(decoder, 
                                                      impute_finished=True, 
                                                      maximum_iterations=max_target_sequence_length)
    return outputs

In [11]:
def decoding_layer(dec_input, encoder_state,
                   target_sequence_length, max_target_sequence_length,
                   rnn_size,
                   num_layers, target_vocab_to_int, target_vocab_size,
                   batch_size, keep_prob, decoding_embedding_size, enc_outputs):
    """
    Create decoding layer
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    target_vocab_size = len(target_vocab_to_int)
    dec_embeddings = tf.Variable(tf.random_uniform([target_vocab_size, decoding_embedding_size]))
    
    
    dec_embed_input = tf.nn.embedding_lookup(dec_embeddings, dec_input)
    #print(dec_embed_input)
    cells = tf.contrib.rnn.MultiRNNCell([tf.contrib.rnn.LSTMCell(rnn_size) for _ in range(num_layers)])
    
    dec_cell = tf.contrib.rnn.DropoutWrapper(cells, output_keep_prob=keep_prob)
    if use_attention :
        attention_mechanism = tf.contrib.seq2seq.BahdanauAttention(
                num_units=batch_size, memory=enc_outputs) 
                #,                memory_sequence_length=150)
        dec_cell = tf.contrib.seq2seq.AttentionWrapper(
                dec_cell, attention_mechanism, attention_layer_size=150)
        dec_initial_state = dec_cell.zero_state(batch_size, tf.float32).clone(cell_state=encoder_state)
    
    with tf.variable_scope("decode"):
        output_layer = tf.layers.Dense(target_vocab_size)
        train_output = decoding_layer_train(dec_initial_state, 
                                            dec_cell, 
                                            dec_embed_input, 
                                            target_sequence_length, 
                                            max_target_sequence_length, 
                                            output_layer, 
                                            keep_prob)

    with tf.variable_scope("decode", reuse=True):
        infer_output = decoding_layer_infer(dec_initial_state, 
                                            dec_cell, 
                                            dec_embeddings, 
                                            target_vocab_to_int['<GO>'], 
                                            target_vocab_to_int['<EOS>'], 
                                            max_target_sequence_length, 
                                            target_vocab_size, 
                                            output_layer,
                                            batch_size,
                                            keep_prob)

    return (train_output, infer_output)

In [12]:
def seq2seq_model(input_data, target_data, keep_prob, batch_size,
                  target_sequence_length,
                  max_target_sentence_length,
                  source_vocab_size, target_vocab_size,
                  enc_embedding_size, dec_embedding_size,
                  rnn_size, num_layers, target_vocab_to_int):
    """
    Build the Sequence-to-Sequence model
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    enc_outputs, (enc_states) = encoding_layer(input_data, 
                                             rnn_size, 
                                             num_layers, 
                                             keep_prob, 
                                             source_vocab_size, 
                                             enc_embedding_size)
    #print(type(enc_states))
    #print(len(enc_states))
    dec_input = process_decoder_input(target_data, 
                                      target_vocab_to_int, 
                                      batch_size)
    
    train_output, infer_output = decoding_layer(dec_input,
                                               enc_states, 
                                               target_sequence_length, 
                                               max_target_sentence_length,
                                               rnn_size,
                                               num_layers,
                                               target_vocab_to_int,
                                               target_vocab_size,
                                               batch_size,
                                               keep_prob,
                                               dec_embedding_size, enc_outputs)
    
    return train_output, infer_output

## Train

In [13]:
#Subroutines used in training
def pad_sentence_batch(sentence_batch, pad_int):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [pad_int] * (max_sentence - len(sentence)) for sentence in sentence_batch]


def get_batches(sources, targets, batch_size, source_pad_int, target_pad_int):
    """Batch targets, sources, and the lengths of their sentences together"""
    for batch_i in range(0, len(sources)//batch_size):
        start_i = batch_i * batch_size

        # Slice the right amount for the batch
        sources_batch = sources[start_i:start_i + batch_size]
        targets_batch = targets[start_i:start_i + batch_size]

        # Pad
        pad_sources_batch = np.array(pad_sentence_batch(sources_batch, source_pad_int))
        pad_targets_batch = np.array(pad_sentence_batch(targets_batch, target_pad_int))

        # Need the lengths for the _lengths parameters
        pad_targets_lengths = []
        for target in pad_targets_batch:
            pad_targets_lengths.append(len(target))

        pad_source_lengths = []
        for source in pad_sources_batch:
            pad_source_lengths.append(len(source))

        yield pad_sources_batch, pad_targets_batch, pad_source_lengths, pad_targets_lengths
        
        
def get_accuracy(target, logits):
    """
    Calculate accuracy
    """
    max_seq = max(target.shape[1], logits.shape[1])
    if max_seq - target.shape[1]:
        target = np.pad(
            target,
            [(0,0),(0,max_seq - target.shape[1])],
            'constant')
    if max_seq - logits.shape[1]:
        logits = np.pad(
            logits,
            [(0,0),(0,max_seq - logits.shape[1])],
            'constant')

    return np.mean(np.equal(target, logits))


#Subrountines used in preparing data for inference. 
def sentence_to_seq(sentence, vocab_to_int):
    results = []
    for word in sentence.split(" "):
        if word in vocab_to_int:
            results.append(vocab_to_int[word])
        else:
            results.append(vocab_to_int['<UNK>'])
            
    return results

In [33]:
#set intervals for peek into the training progress: 
peek_interval=datetime.timedelta(minutes=10)
#Please note peek  may not be on exact above period, because we need to allow each batch to finish. 
#So the batch that finishes 10-min (the period set above) would be where we take a peek

epochs = 10
batch_size = 100
#Add no_peek threshold so system will not peek too frequently on very slow model/systems
only_peek_after = int(len(NYTDATA.source_token_ids)/batch_size/100)

rnn_size = 200
num_layers = 3

encoding_embedding_size = 300
decoding_embedding_size = encoding_embedding_size

use_attention=True
learning_rate = 0.005
keep_probability = 0.95
save_path = "working_dir/"


#NYTDATA.load_preprocess()

max_target_sentence_length = max([len(sentence) for sentence in NYTDATA.source_token_ids])

train_graph = tf.Graph()
with train_graph.as_default():
    input_data, targets, target_sequence_length, max_target_sequence_length = enc_dec_model_inputs()
    
    #lr, keep_prob = hyperparam_inputs()
    lr = tf.placeholder(tf.float32, name='lr_rate')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
  

    train_logits, inference_logits = seq2seq_model(tf.reverse(input_data, [-1]),
                                                   targets,
                                                   keep_prob,
                                                   batch_size,
                                                   target_sequence_length,
                                                   max_target_sequence_length,
                                                   len(NYTDATA.vocab_to_int),
                                                   len(NYTDATA.vocab_to_int),
                                                   encoding_embedding_size,
                                                   decoding_embedding_size,
                                                   rnn_size,
                                                   num_layers,
                                                   NYTDATA.vocab_to_int)
    
    training_logits = tf.identity(train_logits.rnn_output, name='logits')
    inference_logits = tf.identity(inference_logits.sample_id, name='predictions')

    # https://www.tensorflow.org/api_docs/python/tf/sequence_mask
    # - Returns a mask tensor representing the first N positions of each cell.
    masks = tf.sequence_mask(target_sequence_length, max_target_sequence_length, dtype=tf.float32, name='masks')

    with tf.name_scope("optimization"):
        # Loss function - weighted softmax cross entropy
        cost = tf.contrib.seq2seq.sequence_loss(
            training_logits,
            targets,
            masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(lr)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        #capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
        capped_gradients = [(tf.clip_by_norm(grad, 50), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)
print("Graph prepared...", datetime.datetime.now())

Graph prepared... 2018-12-10 19:56:34.193689


In [ ]:
# Split data to training and validation sets
print("Splitting train/valid...", datetime.datetime.now())

train_source = NYTDATA.source_token_ids[batch_size:]
train_target = NYTDATA.target_token_ids[batch_size:]
valid_source = NYTDATA.source_token_ids[:batch_size]
valid_target = NYTDATA.target_token_ids[:batch_size]
(valid_sources_batch, valid_targets_batch, valid_sources_lengths, valid_targets_lengths ) = next(get_batches(valid_source,
                                                                                                             valid_target,
                                                                                                             batch_size,
                                                                                                             NYTDATA.vocab_to_int['<PAD>'],
                                                                                                             NYTDATA.vocab_to_int['<PAD>']))   

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())
    print("start training...", datetime.datetime.now())
    last_peek_time=datetime.datetime.now()
    for epoch_i in range(epochs):
        
        for batch_i, (source_batch, target_batch, sources_lengths, targets_lengths) in enumerate(
            NYTDATA.get_batches(batch_size)):
#                get_batches(train_source, train_target, batch_size,
#                            NYTDATA.vocab_to_int['<PAD>'],
#                            NYTDATA.vocab_to_int['<PAD>'])):
            print(".",end='')
            _, loss = sess.run(
                [train_op, cost],
                {input_data: source_batch,
                 targets: target_batch,
                 lr: learning_rate,
                 target_sequence_length: targets_lengths,
                 keep_prob: keep_probability})

            #check time, and peek into progress if time passed exceeds interval set
            current_time=datetime.datetime.now()
            if (current_time - last_peek_time) > peek_interval :
                last_peek_time=current_time
            #if batch_i % display_step == 0 and batch_i > 0:
                batch_train_logits = sess.run(
                    inference_logits,
                    {input_data: source_batch,
                     target_sequence_length: targets_lengths,
                     keep_prob: 0.95})

                batch_valid_logits = sess.run(
                    inference_logits,
                    {input_data: valid_sources_batch,
                     target_sequence_length: valid_targets_lengths,
                     keep_prob: 0.95})

                train_acc = get_accuracy(target_batch, batch_train_logits)
                valid_acc = get_accuracy(valid_targets_batch, batch_valid_logits)
                
                print(datetime.datetime.now())
                print('\nEpoch {:>3} Batch {:>4}/{} - Train Accuracy: {:>6.4f}, Validation Accuracy: {:>6.4f}, Loss: {:>6.4f}'
                      .format(epoch_i, batch_i, len(NYTDATA.source_token_ids) // batch_size, train_acc, valid_acc, loss))
                print('  source: {}'.format([NYTDATA.int_to_vocab[i] for i in valid_sources_batch[int((1+epoch_i)*(batch_i))%batch_size]]))
                print('  target: {}'.format([NYTDATA.int_to_vocab[i] for i in valid_target_batch[int((1+epoch_i)*(batch_i))%batch_size]]))
                print('  summary: {}'.format([NYTDATA.int_to_vocab[i] for i in batch_valid_logits[int((1+epoch_i)*(batch_i))%batch_size]]))

        # Save checkpoint of training
        saver = tf.train.Saver()
        saver.save(sess, save_path+'ckpt', global_step=epoch_i)
        print('\nModel training checkpoing for epoch_{} is saved'.format(epoch_i))                   
    
    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_path)
    print('Model Trained and Saved')        

Splitting train/valid... 2018-12-10 19:56:34.203219
start training... 2018-12-10 19:56:47.351111
.......................2018-12-10 20:06:58.884493

Epoch   0 Batch   22/983 - Train Accuracy: 0.3878, Validation Accuracy: 0.4006, Loss: 5.5732
  source: ['while', 'standing', 'smooth', 'down', 'as', 'much', 'fabric', 'as', 'possible', 'with', 'both', 'hands', '<p>', 'then', 'cross', 'the', 'knees', 'bend', 'and', 'sit', '<p>', 'optional', 'precaution', 'place', 'purse', 'over', 'lap', '<p>', 'this', 'peculiar', 'choreography', 'is', 'the', 'daily', 'routine', 'for', 'women', 'wearing', 'miniskirts', 'extremely', 'short', 'miniskirts', 'as', 'they', 'try', 'to', 'grab', 'a', 'subway', 'seat', 'with', 'all', 'the', 'grace', 'and', 'modesty', 'they', 'can', 'muster', '<p>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']
  target: ['<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PA

........................2018-12-10 21:39:25.253765

Epoch   0 Batch  237/983 - Train Accuracy: 0.3278, Validation Accuracy: 0.3658, Loss: 4.7344
  source: ['to', 'the', 'editor', 'the', 'use', 'of', 'the', 'confederate', 'battle', 'flag', 'on', 'maryland', 'license', 'plates', 'is', 'not', 'just', 'offensive', 'to', 'many', 'black', 'americans', 'as', 'your', 'dec', '<p>', 'NN', 'news', 'article', 'says', '<p>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']
  target: ['correction', 'of', 'letter', 'on', 'article', 'NN', 'article', 'on', 'article', 'NN', 'article', 'on', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 's', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']
......................2018-12

.......................2018-12-10 23:11:34.723526

Epoch   0 Batch  441/983 - Train Accuracy: 0.3387, Validation Accuracy: 0.3548, Loss: 4.4656
  source: ['to', 'the', 'editor', 'if', 'anything', 'positive', 'is', 'to', 'come', 'of', 'the', 'hostage', 'crisis', 'in', 'peru', 'international', 'attention', 'should', 'focus', 'on', 'the', 'conditions', 'there', 'that', 'led', 'the', 'tupac', 'amaru', 'rebels', 'to', 'such', 'extreme', 'action', '<p>', 'a', 'case', 'in', 'point', 'is', 'the', 'harsh', 'conditions', 'in', 'peru', 's', 'prisons', 'news', 'article', 'dec', '<p>', 'NN', '<p>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']
  target: ['letter', 'from', 'apr', 'NN', 'article', 'on', 'april', 'NN', 'article', 'on', 'april', 'NN', 'article', 'on', 'age', 'NN', 'article', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']
.......................201

...................2018-12-11 00:43:55.568195

Epoch   0 Batch  622/983 - Train Accuracy: 0.2644, Validation Accuracy: 0.2206, Loss: 3.9568
  source: ['in', 'percentpreviousyearyesterdaydayagoprime', 'rateNNNNNNNNNdiscount', 'rateNNNNNNNNNfederal', 'funds', 'NNNNNNNNNN', 'mo', '<p>', 'treas', '<p>', 'billsNNNNNNNNNN', 'mo', '<p>', 'treas', '<p>', 'billsNNNNNNNNNNN', 'yr', '<p>', 'treas', '<p>', 'notesNNNNNNNNNNN', 'yr', '<p>', 'treas', '<p>', 'bondsNNNNNNNNNtelephone', 'bondsNNNNNNNNNmunicipal', 'bonds', 'NNNNNNNNN', 'estimated', 'daily', 'average', 'source', 'telerate', 'municipal', 'bond', 'index', 'the', 'bond', 'buyer', 'salomon', 'brothers', 'and', 'telerate', 'for', 'treasury', 's', 'bellwether', 'bonds', 'notes', 'and', 'bills', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']
  target: ['new', 'york', 'nets', 'of', 'new', 'york', 'city', 'and', 'new', 'york', 'city', 'and', 'new', 'york', 'city', 'and', 'new', 'york', '

........................2018-12-11 02:05:49.032072

Epoch   0 Batch  803/983 - Train Accuracy: 0.2972, Validation Accuracy: 0.3045, Loss: 4.3498
  source: ['peter', 'james', 'roylance', 'a', 'physician', 'and', 'researcher', 'with', 'merck', 'sharp', 'amp', 'dohme', 'a', 'subsidiary', 'of', 'merck', 'amp', 'company', 'the', 'pharmaceutical', 'company', 'died', 'on', 'saturday', 'at', 'his', 'home', 'in', 'long', 'hill', 'township', 'nj', '<p>', 'he', 'was', 'NNhe', 'suffered', 'a', 'long', 'illness', 'his', 'family', 'said', '<p>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']
  target: ['new', 'york', 'mets', 'of', 'new', 'york', 'mets', 'of', 'new', 'york', 'mets', 'in', 'iraq', 'm', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']
..................

...

........................2018-12-12 03:46:13.919228

Epoch   4 Batch  526/983 - Train Accuracy: 0.3013, Validation Accuracy: 0.2584, Loss: 2.9206
  source: ['judging', 'from', 'the', 'way', 'media', 'conglomerates', 'increasingly', 'dominate', 'the', 'magazine', 'industry', 'one', 'might', 'think', 'the', 'idea', 'of', 'starting', 'an', 'independent', 'publication', 'had', 'become', 'a', 'quaint', 'relic', 'of', 'the', 'past', '<p>', 'instead', 'in', 'NNNN', 'the', 'number', 'of', 'new', 'magazines', 'reached', 'a', 'record', 'of', 'NNN', 'and', 'the', 'final', 'tally', 'for', 'NNNN', 'still', 'being', 'calculated', 'is', 'expected', 'to', 'top', 'NNN', '<p>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']
  target: ['article', 'on', 'new', 'york', 'city', 'subway', 'line', 'in', 'new', 'york', 'city', 's', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']
....

........................2018-12-12 05:17:04.496735

Epoch   4 Batch  742/983 - Train Accuracy: 0.2978, Validation Accuracy: 0.3203, Loss: 3.0276
  source: ['the', 'duke', 'and', 'duchess', 'of', 'kent', 'are', 'grandparents', 'once', 'again', '<p>', 'lady', 'helen', 'taylor', 'their', 'daughter', 'and', 'her', 'husband', 'tim', 'taylor', 'an', 'art', 'dealer', 'have', 'a', 'second', 'son', '<p>', 'the', 'child', 'born', 'the', 'day', 'after', 'christmas', 'is', 'NNth', 'in', 'line', 'to', 'the', 'british', 'throne', '<p>', 'should', 'he', 'defy', 'all', 'odds', 'and', 'ascend', 'taking', 'his', 'christian', 'name', 'with', 'him', 'he', 'would', 'be', 'king', 'cassius', '<p>']
  target: ['jack', 'anderson', 'reviews', 'dance', 'theater', 'production', 'of', 'molissa', 'bagnasco', 'play', 'by', 'john', 'hubley', 'directed', 'by', 'robert', 'smihula', 'm', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']
.................

........................2018-12-12 06:47:52.968822

Epoch   4 Batch  958/983 - Train Accuracy: 0.3956, Validation Accuracy: 0.2952, Loss: 2.9505
  source: ['new', 'hope', 'inc', 'maitland', 'fla', 'to', 'cramer', 'krasselt', 'orlando', 'fla', 'as', 'the', 'first', 'agency', 'to', 'handle', 'advertising', 'and', 'public', 'relations', 'for', 'the', 'nonprofit', 'organization', 'that', 'offers', 'crisis', 'assistance', '<p>', 'billings', 'were', 'not', 'disclosed', '<p>', 'ascent', 'pediatrics', 'inc', 'billerica', 'mass', 'to', 'integrated', 'communications', 'corp', 'langhorne', 'pa', 'as', 'the', 'first', 'agency', 'for', 'the', 'company', 'which', 'sells', 'children', 's', 'medicine', '<p>', 'billings', 'were', 'not', 'disclosed', '<p>', '<PAD>']
  target: ['roundup', 'of', 'recent', 'advertising', 'account', 'changes', 's', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>',

........................2018-12-12 08:19:00.273266

Epoch   5 Batch  192/983 - Train Accuracy: 0.3787, Validation Accuracy: 0.3400, Loss: 2.9881
  source: ['singapore', 's', 'economy', 'expanded', 'NN', 'percent', 'in', 'NNNN', 'the', 'prime', 'minister', 'said', 'yesterday', '<p>', 'the', 'rate', 'of', 'growth', 'was', 'the', 'slowest', 'pace', 'since', 'NNNN', 'when', 'gross', 'domestic', 'product', 'gained', 'NN', 'percent', 'from', 'the', 'previous', 'year', '<p>', 'the', 'figures', 'are', 'adjusted', 'for', 'inflation', '<p>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']
  target: ['oil', 'prices', 'rise', 'NN', 'cents', 'to', 'NNNN', 'a', 'barrel', 'dollar', 'rises', 'NN', 'cents', 'to', 'NNNN', 'cents', 'from', 'NNNNN', 'graph', 'chart', 's', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']
......

........................2018-12-12 09:50:26.592998

Epoch   5 Batch  402/983 - Train Accuracy: 0.4022, Validation Accuracy: 0.3558, Loss: 3.0115
  source: ['the', 'holidays', 'are', 'over', 'the', 'gifts', 'given', 'and', 'received', 'the', 'family', 'snapshots', 'taken', '<p>', 'now', 'comes', 'the', 'tricky', 'part', 'getting', 'the', 'film', 'processed', 'properly', 'and', 'storing', 'the', 'pictures', 'so', 'that', 'they', 'will', 'last', 'as', 'long', 'as', 'possible', '<p>', 'most', 'people', 'learn', 'the', 'importance', 'of', 'these', 'tasks', 'the', 'hard', 'way', 'by', 'watching', 'a', 'treasured', 'color', 'snapshot', 'fade', 'to', 'purple', '<p>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']
  target: ['article', 'on', 'cooking', 'cooking', 'cooking', 'holiday', 'decorations', 'in', 'new', 'york', 'city', 'photo', 'm', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PA

## inference

In [ ]:
#with open('working_dir/100words_or_less_preprocess.p', 'rb') as f:
#            NYTDATA = pickle.load(f)
#batch_size=100 

In [ ]:
#test inference_logits
   
test_sentences_raw = [' '.join(shortern.split()) for shortern in NYTDATA.source_raw[0:0+batch_size]]
print(len(test_sentences_raw))

test_target_tokenized = [sent_tokenize_rm_punct(' '.join(shortern.split())) for shortern in NYTDATA.source_raw[0:0+batch_size]]
print(len(test_target_tokenized))

test_sentences = NYTDATA.infer_prep(test_sentences_raw, pad=True)[0:batch_size]
print(len(test_sentences))
test_sentence_length = [len(ts) for ts in test_sentences]   


In [ ]:
loaded_graph = tf.Graph()

with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    checkpoint_file=save_path+'ckpt-3'
    loader = tf.train.import_meta_graph(checkpoint_file + '.meta')
    loader.restore(sess, checkpoint_file)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    inference_logits = loaded_graph.get_tensor_by_name('predictions:0')
    target_sequence_length = loaded_graph.get_tensor_by_name('target_sequence_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')

    
    translate_logits = sess.run(inference_logits, {input_data: test_sentences,
                                         target_sequence_length: test_sentence_length,
                                         keep_prob: 0.95})
    for ii in range(len(test_sentences)) :
        print('\nRaw: {}'.format(test_sentences_raw[ii]))
        #print('  Word Ids:      {}'.format([i for i in test_sentences[ii]]))
        print('\nInput: {}'.format([NYTDATA.int_to_vocab[i] for i in test_sentences[ii] if i > 0]))

        #print('\nOutput')
        #print('  Word Ids:      {}'.format([i for i in translate_logits[ii]]))
        print('  \nOutput: {}\n'.format(" ".join([NYTDATA.int_to_vocab[i] for i in translate_logits[ii] if i>0])))
    #finished test infer
    

In [ ]:
import rouge
hypotheses=[]
references=test_target_tokenized
for ii in range(len(test_sentences)):
    hypotheses.append(" ".join([NYTDATA.int_to_vocab[i] for i in translate_logits[ii] if i > 0 ]))

rouge_score_map = rouge.rouge(hypotheses, references)
print("ROUGE_1:{}".format(100 * rouge_score_map["rouge_1/f_score"]))
print("ROUGE_2:{}".format(100 * rouge_score_map["rouge_2/f_score"]))
print("ROUGE_l:{}".format(100 * rouge_score_map["rouge_l/f_score"]))


In [ ]:
#print(test_sentences[:][0:100])
print([len(shorten[0:100]) for shorten in NYTDATA2.target_raw[0:0+batch_size]])

In [ ]:
print(len(NYTDATA2.vocab_to_int))

In [ ]:
templ = [tt for tt in NYTDATA.source_token_ids if 332197 in tt]
print(templ)
for tempi in templ :
    print([NYTDATA.int_to_vocab[t] for t in tempi])

In [ ]:
print(NYTDATA.vocab_to_int['<PAD>'])
print(NYTDATA.vocab_to_int['<EOS>'])
print(NYTDATA.vocab_to_int['<UNK>'])
print(NYTDATA.vocab_to_int['<GO>'])
print(NYTDATA.vocab_to_int['<p>'])
print(NYTDATA.int_to_vocab[4])

#{'<PAD>': 0, '<EOS>': 1, '<UNK>': 2, '<GO>': 3 , '<p>': 4}

In [ ]:
NYTDATA.source_token_ids=NYTDATA.target_token_ids